In [1]:
# 1-2.py
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# .env 파일 로드
load_dotenv()

# Groq API를 사용하는 ChatOpenAI 인스턴스 생성 (이전 노트북에서 사용된 방식 그대로)
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct", # 또는 다른 Groq 모델
    temperature=0.7
)

# 1단계 체인: 영화 추천
# ChatPromptTemplate 사용: system과 human 메시지 지정
prompt_stage1 = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 추천 전문가입니다. 사용자가 선호하는 장르에 맞춰 한 편의 영화를 추천하고, 그 영화에 대한 간단한 설명을 덧붙여 주세요."),
    ("human", "{genre} 장르에서 재미있는 영화를 한 편 추천해주고, 그 영화의 제목만 먼저 알려주세요. 그 다음 줄을 바꾸어 간단한 설명을 덧붙여주세요.")
])

# 2단계 체인: 줄거리 요약
# ChatPromptTemplate 사용: system과 human 메시지 지정
prompt_stage2 = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 줄거리 요약 전문가입니다. 제공된 영화의 제목을 먼저 명시하고, 그 영화의 줄거리를 3문장으로 간결하게 요약해 주세요."),
    ("human", "{movie_title} 영화의 줄거리를 3문장으로 요약해 주세요.")
])

# Output Parser
output_parser = StrOutputParser()

# 체인 1 정의: 장르 -> 영화 추천 (제목과 간단 설명)
chain1 = prompt_stage1 | llm | output_parser

# 체인 2 정의: 추천 영화 제목 -> 줄거리 요약
# chain1의 출력을 movie_title 변수로 전달
chain2 = (
    {"movie_title": chain1}  # chain1의 출력을 'movie_title' 입력으로 사용
    | prompt_stage2
    | llm
    | output_parser
)

# 실행: "액션" 장르의 영화 추천 및 줄거리 요약
user_genre = "액션"
print(f"입력: \"{user_genre}\"")

try:
    # 1단계 체인 결과 확인
    stage1_result = chain1.invoke({"genre": user_genre})
    print(f"1단계 결과: \"{stage1_result.strip()}\"") # 공백 제거하여 깔끔하게 출력

    # 2단계 체인 결과 확인
    stage2_result = chain2.invoke({"genre": user_genre}) # chain2는 내부적으로 chain1을 호출
    print(f"2단계 결과:\n{stage2_result.strip()}") # 공백 제거하여 깔끔하게 출력

except Exception as e:
    print(f"오류 발생: {e}")

입력: "액션"
1단계 결과: "영화 제목: 존 윅

이 영화는 전직 암살자 존 윅이 은퇴한 후, 범죄 조직의 보스에게서 강아지를 선물로 받지만, 그 강아지가 살해되면서 시작되는 복수극입니다. 액션과 스토리, 연기가 모두 훌륭하여 많은 팬을 보유하고 있는 영화입니다."
2단계 결과:
영화 제목: 존 윅

전설적인 킬러 존 윅은 은퇴 후 평화로운 삶을 살고자 하지만, 우연히 범죄 조직의 표적이 되어 목숨을 위협받게 됩니다. 존 윅은 자신을 죽이기 위해 파견된 수많은 킬러들을 상대로 목숨을 건 싸움을 시작합니다. 이 과정에서 그는 자신의 과거와 조직의 비밀에 대해 조금씩 깨닫게 되며, 살아남기 위한 치열한 생존 싸움을 벌입니다.
